In [37]:
%matplotlib inline
import numpy as np
import pandas as pd
import time

In [38]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [39]:
votes = pd.read_csv("../raw/train_data.csv")

In [40]:
answers = pd.read_csv("../raw/train_answers.csv")

In [41]:
votes_test = pd.read_csv("../raw/test_data.csv")

In [42]:
data = votes.merge(answers, on="itemId").set_index("itemId")
data.head()

,userId,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true
itemId,,,,,,,,,
6132,1164,62,656,276,803,47,650,213,812
6132,1399,184,662,235,740,47,650,213,812
6132,1694,63,648,272,830,47,650,213,812
6132,916,111,637,253,831,47,650,213,812
6132,1621,60,651,242,819,47,650,213,812


In [43]:
data_values = data.drop("userId", axis=1).groupby("itemId").median().values
errors = data_values[:,4:] - data_values[:,:4]
pd.DataFrame(errors).describe()

,0,1,2,3
count,943.000000,943.000000,943.000000,943.000000
mean,-23.108165,-14.727466,22.009544,12.149523
std,86.789074,57.642590,97.963222,94.113843
min,-511.500000,-355.000000,-259.000000,-351.500000
25%,-59.250000,-37.500000,-21.000000,-26.000000
50%,-14.000000,-8.000000,11.000000,6.000000
75%,11.750000,8.000000,66.000000,50.500000
max,287.000000,185.000000,466.000000,365.000000


In [46]:
sub = votes_test.drop("userId", axis=1).groupby("itemId").median()
sub["Xmin"] -= 14
sub["Ymin"] -= 8
sub["Xmax"] += 11
sub["Ymax"] += 6

In [47]:
sub.to_csv(
    f"../processed/sub_{time.strftime('%Y-%m-%d_%H-%M')}.csv",
    header=False
)